# Google Play Store Data Analytics Project


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import nltk
import webbrowser
import os                                                                     

In [2]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sravy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
apps_df = pd.read_csv("Play Store Data.csv")
reviews_df=pd.read_csv("User Reviews.csv")

In [4]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [6]:
#step2 : Data Cleaning
apps_df = apps_df.dropna(subset=["Rating"])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df=apps_df[apps_df["Rating"]<=5]
reviews_df.dropna(subset=["Translated_Review"],inplace=True)

C:\Users\sravy\AppData\Local\Temp\ipykernel_19476\3977620194.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)


In [7]:
#convert the Installs columns to numeric by removing commas and +
apps_df["Installs"]=apps_df["Installs"].str.replace(',','').str.replace('+','').astype(int)

#Convert price column to numeric after removing $
apps_df["Price"]=apps_df["Price"].str.replace('$','').astype(float)

In [8]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [9]:
merged_df=pd.merge(apps_df,reviews_df,on="App",how="inner")

In [10]:
merged_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


In [11]:
#Data Transformation
apps_df["Reviews"]=apps_df["Reviews"].astype(int)

In [12]:
def convert_size(size):
    if"M" in size:
        return float(size.replace("M",''))
    elif "k" in size:
        return float(size.replace('k',''))/1024
    else:
        return np.nan
apps_df["Size"]=apps_df["Size"].apply(convert_size)
    

In [13]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [14]:
#Lograrithmic
apps_df["Log_Installs"]=np.log1p(apps_df["Installs"])


In [15]:
apps_df["Reviews"]=apps_df["Reviews"].astype(int)

In [16]:
apps_df["Log_Reviews"]=np.log1p(apps_df["Reviews"])


In [17]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews             int32
Size              float64
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
Log_Installs      float64
Log_Reviews       float64
dtype: object

In [18]:
def rating_group(rating):
    if rating>=4:
        return "Top rated app"
    elif rating >=3:
        return "Above average"
    else:
        retuen:"Below Average"
apps_df["Rating_Group"]=apps_df["Rating"].apply(rating_group)

In [19]:
apps_df["Rating_Group"]

0        Top rated app
1        Above average
2        Top rated app
3        Top rated app
4        Top rated app
             ...      
10834    Top rated app
10836    Top rated app
10837    Top rated app
10839    Top rated app
10840    Top rated app
Name: Rating_Group, Length: 8892, dtype: object

In [20]:
#Revenue column
apps_df["Revenue"]=apps_df["Price"]*apps_df["Installs"]
apps_df["Revenue"]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10834    0.0
10836    0.0
10837    0.0
10839    0.0
10840    0.0
Name: Revenue, Length: 8892, dtype: float64

In [21]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [22]:
#Polarity Scores in SIA
#Positive, Negative, Neutral and Compound: -1 - Very negative; +1 - very positive

In [23]:
review = "This app is amazing! I Love the new features."
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.458, 'pos': 0.542, 'compound': 0.8516}


In [24]:
review = "This app is very bad! I hate the new features."
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.502, 'neu': 0.498, 'pos': 0.0, 'compound': -0.8422}


In [25]:
review = "This app is okey."
sentiment_score=sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [26]:
reviews_df["Sentiment_Score"]=reviews_df["Translated_Review"].apply(lambda x: sia.polarity_scores(str(x))["compound"])

In [27]:
reviews_df.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333,0.9531
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462,0.6597
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000,0.6249
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000,0.6369
5,10 Best Foods for You,Best way,Positive,1.00,0.300000,0.6369


In [28]:
apps_df["Last Updated"]=pd.to_datetime(apps_df["Last Updated"],errors="coerce")

In [29]:
apps_df["Year"]=apps_df["Last Updated"].dt.year

In [30]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_Group,Revenue,Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,9.210440,5.075174,Top rated app,0.0,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,13.122365,6.875232,Above average,0.0,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,15.424949,11.379520,Top rated app,0.0,2018
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,17.727534,12.281389,Top rated app,0.0,2018
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,11.512935,6.875232,Top rated app,0.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,2017-06-18,1.0.0,4.1 and up,6.216606,2.079442,Top rated app,0.0,2017
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,2017-07-25,1.48,4.1 and up,8.517393,3.663562,Top rated app,0.0,2017
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,2018-07-06,1.0,4.1 and up,4.615121,1.609438,Top rated app,0.0,2018
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,2015-01-19,Varies with device,Varies with device,6.908755,4.744932,Top rated app,0.0,2015


In [31]:
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)

In [32]:
plot_containers=""

In [33]:
def save_plot_as_html(fig,filename,insight):
    global plot_containers
    filepath = os.path.join(html_files_path,filename)
    html_content=pio.to_html(fig,full_html=False, include_plotlyjs="inline")
    plot_containers+=f"""
    <div class = "plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div> 
        <div class="insights">{insight}</div> 
    </div>"""
    fig.write_html(filepath, full_html=False, include_plotlyjs="inline")
    

In [34]:
plot_width = 400
plot_height=300
plot_bg_color="black"
text_color="white"
title_font={"size":16}
axis_font={"size":12}

# Plotly graphs

In [54]:
#Figure 1
category_counts=apps_df["Category"].value_counts().nlargest(10)
fig1=px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={"x": "Category", "y":"Count"},
    title="Top Categories on Play Store",
    color=category_counts.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)
fig1.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig1,"Category Graph 1.html", "The top Categories on the play store are dominated by tools, entertainment, and productivity apps")

In [55]:
#Figure 2
type_counts=apps_df["Type"].value_counts()
fig2=px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title="App Type Distribution",
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
fig1.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig2.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig2,"Type Graph 2.html", "Most apps on the Playstore are free, indicating a strategy to attract users first and monetize through ads or in app purchases")

In [56]:
#Figure 3

fig3=px.histogram(
    apps_df,
    x="Rating",
    nbins=20,
    title="Rating Distribution",
    color_discrete_sequence=["#636EFA"],
    width=400,
    height=300
)
fig3.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)
#fig1.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig3,"Rating Graph 3.html","Ratings are skewed towards higher values, suggesting that most apps are rated favorably by users") 

In [57]:
#Figure 4
sentiment_counts=reviews_df["Sentiment_Score"].value_counts()
fig4=px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={"x":"Sentiment Score","y":"count"},
    title="Sentiment Distribution",
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)
fig4.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig4.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig4,"Sentiment Graph 4.html","Sentiments in reviews show a mix of positive and negative feedback, with a slight lean towards postive sentiments") 
    
    

In [58]:
#Figure 5
installs_by_category=apps_df.groupby("Category")["Installs"].sum().nlargest(10)
fig5=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    orientation="h",
    labels={"x":"Installs","y":"Category"},
    title="Installs by Category",
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)
fig5.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig5.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig5,"Installs Graph 5.html","The categories with the most installs are social and communication apps, reflecting their broad appeal and daily usage ") 
    
    

In [40]:
#Updates Per Year Plot
updates_per_year=apps_df["Last Updated"].dt.year.value_counts().sort_index()
fig6 = px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={"x":"Year","y":"Number of Updates"},
    title="Number of Updates Over the Years",
    color_discrete_sequence=["#AB63FA"],
    width=plot_width,
    height=plot_height
)
fig6.update_layout(
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=plot_bg_color,
    font_color=text_color,
    title_font= title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig6.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig6,"Updates Graph 6.html","Updates have been increasing over the years, showing that developers are actively maintaining and improving their apps.") 
     

In [41]:
#Figure 7
revenue_by_category=apps_df.groupby("Category")["Revenue"].sum().nlargest(10)
fig7=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    labels={"x":"Category","y":"Revenue"},
    title="Revenue by Category",
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)
fig7.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig7.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig7,"Revenue Graph 7.html","categories such as Business and Productivity lead in Revenue generation, indicating their monetization potential   ") 
    

In [42]:
#Figure 8
genre_counts=apps_df["Genres"].str.split(';',expand=True).stack().value_counts().nlargest(10)
fig8=px.bar(
    x=installs_by_category.index,
    y=installs_by_category.values,
    labels={"x": "Genre","y":"Count"},
    title="Top Genres",
    color=installs_by_category.index,
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)
fig8.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig5.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig8,"Genre Graph 8.html"," Action and Casual genres are the most common, reflecting users preference for engaging and easy-to-play games ") 
    

In [43]:
#Figure 9
fig9=px.scatter(
    apps_df,
    x='Last Updated',
    y='Rating',
    title="Impact of Last Update on Rating",
    color='Type',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig9.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig9,"Update Graph 9.html","The Scatter Plot shows a weak correction between the last update and ratings, suggesting that more frequent updates don't always result in better ratings") 
    

In [44]:
#Figure 10
fig10=px.box(
    apps_df,
    x='Type',
    y='Rating',
    title="Rating for Paid vs Free Apps",
    color='Type',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor="black",
    paper_bgcolor="black",
    font_color="white",
    title_font={"size":16},
    xaxis=dict(title_font={"size":12}),
    yaxis=dict(title_font={"size":12}),
    margin=dict(l=10,r=10,t=30,b=10)
)

#fig9.update_traces(marker=dict(marker=dict(line=dict(color="white",width=1))))
save_plot_as_html(fig10,"Paid Free Graph 10.html","Paid apps generally have higher ratings compared to free apps, suggesting that users expect higher quality from apps they pay for") 
    

In [45]:
plot_containers_split=plot_containers.split('</div>')

In [46]:
if len(plot_containers_split) > 1:
    final_plot= plot_containers_split[-2]+'</div>'
else:
    final_plot = plot_containers

In [67]:
dashboard_html="""
<!DOCTYPE html>
<html lang="en"
<head>
    <meta charset="UTF-8">
    <meta name=viewport" content="width=device-width, initial-scale-1.0">
    <title> Google Play Store Review Analytics </title>
    <style>
        body{{
             font-family: Arial, sans-serif;
             background-color: #333;
             color: #fff;
             margin: 0;
             padding: 0;
            }}
            .header{{
                display: flex;
                align-items: center;
                justify-content:center;
                padding: 20px;
                background-color: #444

            }}
            .header img {{
                margin: 0 10 px;
                height: 50px;
            }}
             .container{{
                 display: flex;
                 flex-wrap: wrap;
                 justify_content: center;
                 padding: 20px;
            }}
            .plot-container {{
                 border:2px solid #555
                 margin: 10px;
                 padding: 10px;
                 width: {plot_width}px;
                 height: {plot_height}px;
                 overflow: hidden;
                 position: relative;
                 cursor: pointer;
            }}
            .insights{{
                display: none;
                position: absolute;
                right: 10px;
                top: 10px;
                background-color: rgba(0,0,0,0.7);
                padding: 5px;
                border-radius: 5px;
                color: #fff;
            }}
                
            .plot-container: hover .insights {{
                  display: block;
            }}
            </style>
            <script>
                function openPlot(filename) {{
                   window.open(filename, "_blank");
                   }}
           </script>
    </head>
    <body>
         <div class "header>
            <img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
            <h1> Google Play Store Reviews Analytics </h1>
            <img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Logo">
      </div>
      <div class ="container">
          {plots}
      </div>
   </body>
</html>
 """
    

In [68]:
#use these containers to fill your dashboard HTML
final_html=dashboard_html.format(plots=plot_containers, plot_width=plot_width,plot_height=plot_height)

In [69]:
#Save the final dashboard to an HTML file
dashboard_path = os.path.join(html_files_path,"web page.html")
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(final_html)

In [70]:
#Automatically open the generated HTML file in a web browser
webbrowser.open("file://"+os.path.realpath(dashboard_path))

True